In [1]:
import numpy as np
import pandas as pd
import scipy 
import cufflinks as cf
import seaborn as sns
%matplotlib inline
from dask import dataframe

In [3]:
train_df = pd.read_csv('train.csv')

In [8]:
train_df.shape

(74180464, 11)

In [18]:
client_df = pd.read_csv('cliente_tabla.csv')
product_df = pd.read_csv('producto_tabla.csv')
town_df = pd.read_csv('town_state.csv')

In [20]:
train_df_ext = pd.merge(train_df, town_df, left_on='Agencia_ID', right_on='Agencia_ID', how='outer')

In [28]:
train_df_ext.columns

Index([u'Semana', u'Agencia_ID', u'Canal_ID', u'Ruta_SAK', u'Cliente_ID',
       u'Producto_ID', u'Venta_uni_hoy', u'Venta_hoy', u'Dev_uni_proxima',
       u'Dev_proxima', u'Demanda_uni_equil', u'Town', u'State'],
      dtype='object')

In [31]:
train_df_ext['Q']=pd.qcut(train_df_ext.Demanda_uni_equil,5)


In [32]:
for col in train_df.columns:
    a = train_df[col].unique()
    b = a.shape[0]
    if b > 100:
        
        print 'The values of column: {0} are: {1}'.format(col,b)
    else:
        print 'The values of column: {0} are: {1}'.format(col, a)

The values of column: Semana are: [3 4 5 6 7 8 9]
The values of column: Agencia_ID are: 552
The values of column: Canal_ID are: [ 7  8 11  1  4  2  6  5  9]
The values of column: Ruta_SAK are: 3603
The values of column: Cliente_ID are: 880604
The values of column: Producto_ID are: 1799
The values of column: Venta_uni_hoy are: 2116
The values of column: Venta_hoy are: 78140
The values of column: Dev_uni_proxima are: 558
The values of column: Dev_proxima are: 14707
The values of column: Demanda_uni_equil are: 2091


<h2> Find the number of clients per state </h2> 

In [34]:
aggregations = {'Cliente_ID': {'unique_clinets': lambda x: len(set(x)),
                                'clients_per_state': lambda x: x.shape[0]}}
per_state = train_df_ext.groupby(['State']).agg(aggregations)

In [39]:
per_state.loc[slice(None), ('Cliente_ID','unique_clinets')].sum()

886785.0

<p> The total unique clients in this dataset is 880604, interestingly, it seems that clients who are somehow seperable by state as the sum of unique counts of clients per state ~= 886785, counts of unique clients. Lets check that by investigating how many times the clients exists in each state

In [85]:
aggregations = {'State': {'count': lambda x: x.shape[0], 'unique_counts': lambda x: len(set(x))},
               'Producto_ID':{'count': lambda x: x.shape[0], 'unique_counts': lambda x: len(set(x))}}
counts_per_clinet = train_df_ext.groupby(['Cliente_ID']).agg(aggregations)

In [88]:
counts_per_clinet.sort_values(by=('State','unique_counts'), ascending=False, inplace=True)

In [90]:
counts_per_clinet.head()

State               Producto_ID              
             count unique_counts       count unique_counts
Cliente_ID                                                
653378.0    124059            30    124059.0         212.0
933857.0        87             3        87.0           5.0
1596201.0      296             3       296.0          73.0
1216931.0      529             3       529.0          61.0
1118103.0      401             3       401.0          84.0

<p>
What for an outlier, a single client who is spread across the whole country, how many products he serves?
</p>

In [95]:
train_df_ext[train_df_ext.Cliente_ID==653378.0].Producto_ID.nunique()

212

<p> is the the biggest client? </p>

In [97]:
train_df_ext.groupby('Cliente_ID')['Producto_ID'].nunique().sort_values(ascending=False).head(10)

Cliente_ID
2191379.0    215
653378.0     212
1483356.0    199
2460792.0    199
1327215.0    198
4660866.0    193
4686785.0    191
157682.0     190
956730.0     189
2287463.0    185
Name: Producto_ID, dtype: int64

interesting, the variance of the product variay of clients products is small compared to their geographical span!!
Lets see the 2191379.0

In [99]:
print counts_per_clinet.ix[2191379.0]
print counts_per_clinet.ix[653378.0]

State        count            1121.0
             unique_counts       2.0
Producto_ID  count            1121.0
             unique_counts     215.0
Name: 2191379.0, dtype: float64
State        count            124059.0
             unique_counts        30.0
Producto_ID  count            124059.0
             unique_counts       212.0
Name: 653378.0, dtype: float64


OK! not that much interesting. only 2 locations with 1121 products. Now we come with the next question? Is the the  count of products exists in the quantity the client asks for? To investigate that, we will take two clients: 2191379. and 653378. Take a product_id as index with the maximum entries. take the quantity as column and see ...

In [100]:
# only 2 clients 
clients =[653378.0, 2191379.0]
sub_train = train_df_ext[train_df_ext.Cliente_ID.isin(clients)]

In [107]:
# take the product with maximum count from 2191379
counts_per_prod = sub_train.groupby(['Cliente_ID', 'Producto_ID'])['Semana'].count().unstack(level='Cliente_ID')
counts_per_prod['sum'] = counts_per_prod[653378.0] + counts_per_prod[2191379.0]
res = counts_per_prod.dropna(inplace=False)
res

Cliente_ID,653378.0,2191379.0,sum
Producto_ID,,,
1216.0,69.0,7.0,76.0
1220.0,504.0,7.0,511.0
1240.0,1815.0,7.0,1822.0
1242.0,252.0,7.0,259.0
1250.0,1873.0,7.0,1880.0
1284.0,184.0,7.0,191.0
1309.0,1025.0,7.0,1032.0
3526.0,55.0,2.0,57.0
4280.0,1769.0,14.0,1783.0


In [120]:
# get the average quantity of each product for those two: 653378.0, 2191379.0
aggregations = {'Demanda_uni_equil':{'max': 'max',
                                    'min': 'min',
                                    'median': 'median' }}
stat_prod_order = sub_train.groupby(['Cliente_ID', 'Producto_ID']).agg(aggregations).unstack(level='Cliente_ID')
                

In [121]:
stat_prod_order.ix[res.index]

Demanda_uni_equil                                          \
                          max              median                 min   
Cliente_ID          653378.0  2191379.0 653378.0  2191379.0 653378.0    
Producto_ID                                                             
1216.0                  108.0     150.0      37.0      83.0       0.0   
1220.0                  320.0     139.0      86.0      75.0       0.0   
1240.0                  759.0     161.0     224.0     102.0       0.0   
1242.0                  330.0     190.0      57.0     139.0       0.0   
1250.0                 1203.0     276.0     344.0     178.0       0.0   
1284.0                  523.0     123.0     138.5      43.0       0.0   
1309.0                  312.0      36.0      78.0      30.0       0.0   
3526.0                  200.0      48.0      85.0      32.0       0.0   
4280.0                  464.0      48.0     175.0      24.0       0.0   
5337.0                  135.0      12.0      40.0       4.0       0.0   
5380.0                  723.0      29.0     175.0      15.0       0.0   
30549.0                 296.0      18.0     101.0       5.0       0.0   
30552.0                 310.0      25.0      87.5       8.0       0.0   
30555.0                 294.0      26.0      48.0      10.0       0.0   
30568.0                 319.0      14.0      52.0       3.0       0.0   
30569.0                 515.0      33.0      67.0       9.0       0.0   
30572.0                 610.0      13.0     123.0       7.0       0.0   
30575.0                 393.0      24.0      76.0      10.0       0.0   
31309.0                 405.0      42.0     163.0      42.0       0.0   
31506.0                 512.0      42.0     217.0      42.0      82.0   
32393.0                1428.0      65.0     226.0       7.0       0.0   
32846.0                 256.0      18.0     117.0       6.0       0.0   
32861.0                 422.0       5.0     117.0       2.5       0.0   
33871.0                 270.0     542.0      19.0      22.0       0.0   
34053.0                  41.0     164.0       3.5      34.0       0.0   
34056.0                 247.0      29.0      45.0      19.0       0.0   
35107.0                 327.0       3.0      80.5       3.0       0.0   
35516.0                  46.0      10.0       8.0       5.0       0.0   
36008.0                 259.0      30.0      51.0      17.0       0.0   
37057.0                 404.0      19.0     152.0       9.0       0.0   
37058.0                 569.0     183.0     144.0     123.0       0.0   
40985.0                1139.0      12.0     337.5      12.0       0.0   
41843.0                 382.0      27.0      47.0      27.0       0.0   
42434.0                 253.0      35.0      58.0      13.0       0.0   
42869.0                 365.0      15.0      96.0       9.0       0.0   
43033.0                  39.0      16.0      39.0      13.0      39.0   
43064.0                 204.0      99.0      60.5      68.0       0.0   
43068.0                 178.0      49.0      85.0      23.0       0.0   
43069.0                 667.0     135.0     187.0      64.0       0.0   
43274.0                 349.0      54.0      75.0      30.0       0.0   
48417.0                 154.0      43.0      72.0      17.0       0.0   
48893.0                 971.0       1.0     143.0       1.0       0.0   

                       
                       
Cliente_ID  2191379.0  
Producto_ID            
1216.0           69.0  
1220.0           58.0  
1240.0           80.0  
1242.0          111.0  
1250.0          141.0  
1284.0           20.0  
1309.0           17.0  
3526.0           16.0  
4280.0            3.0  
5337.0            2.0  
5380.0           11.0  
30549.0           2.0  
30552.0           4.0  
30555.0           2.0  
30568.0           2.0  
30569.0           6.0  
30572.0           3.0  
30575.0           1.0  
31309.0          42.0  
31506.0          42.0  
32393.0           1.0  
32846.0           2.0  
32861.0           1.0  
33871

Can we use population of states as features? Lets first collect them

In [130]:
states = pd.Series({'ESTADO DE M\xc3\x89XICO':16187608,
'M\xc3\x89XICO, D.F.':8918653,
'VERACRUZ':8112505,
'JALISCO':7844830,
'PUEBLA':6168883,
'GUANAJUATO':5853677,
'CHIAPAS':5217908,
'NUEVO LE\xc3\x93N':5119504,
'MICHOAC\xc3\x81N':4584471,
'OAXACA':3967889,
'CHIHUAHUA':3556574,
'GUERRERO':3533251,
'TAMAULIPAS':3441698,
'BAJA CALIFORNIA':3315766,
'SINALOA':2966321,
'COAHUILA':2954915,
'HIDALGO':2858359,
'SONORA':2850330,
'SAN LUIS POTOS\xc3\x8d':2717820,
'TABASCO':2395272,
'YUCAT\xc3\x81N':2097175,
'QUERETARO':2038372,
'MORELOS':1903811,
'DURANGO':1754754,
'ZACATECAS':1579209,
'QUINTANA ROO':1501562,
'AGUASCALIENTES':1312544,
'TLAXCALA':1272847,
'NAYARIT':1181050,
'CAMPECHE':899931,
'BAJA CALIFORNIA SUR':712029,
'COLIMA':711235})
states = states.reset_index()
states.columns = ['state', 'population']

In [134]:
train_df_ext_pop = pd.merge(train_df_ext, states, left_on='State', right_on='state', how='outer')

In [139]:
train_df_ext_pop.drop('state', axis=1, inplace=True)

In [144]:
train_df_ext_pop.to_hdf('train_df_ext', 'train', mode='a')

AttributeError: 'module' object has no attribute 'open_file'